In [1]:
# === SEKCJA 1: IMPORT I KONFIGURACJA ===
import pandas as pd
import numpy as np
import mlflow
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from pycaret.classification import setup, compare_models, tune_model, finalize_model, save_model, pull
from IPython.display import display

MLFLOW_EXPERIMENT_NAME = 'Warsaw_Street_Prediction_FINAL'
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)
print(f"MLflow ustawiony. Eksperyment: '{MLFLOW_EXPERIMENT_NAME}'")

print("\nWczytywanie danych...")
df_main = pd.read_csv('data.csv', sep=',')
df_ulic = pd.read_csv('Ulic.csv', sep=',') # Wczytujemy poprawny plik z ulicami
print(f"Wczytano dane główne: {df_main.shape}, Słownik ulic: {df_ulic.shape}")

2025/06/16 16:48:27 INFO mlflow.tracking.fluent: Experiment with name 'Warsaw_Street_Prediction_FINAL' does not exist. Creating a new experiment.


MLflow ustawiony. Eksperyment: 'Warsaw_Street_Prediction_FINAL'

Wczytywanie danych...
Wczytano dane główne: (196240, 51), Słownik ulic: (293077, 9)


In [2]:
df_pom_1.info()

NameError: name 'df_pom_1' is not defined

In [ ]:
df_pom_1.head()

In [3]:
# === SEKCJA 2: PRZYGOTOWANIE I WZBOGACANIE DANYCH ===

# --- Krok 2a: Stworzenie słownika ulic z pliku Ulic.csv ---
# Kluczem będzie symbol ulicy (SYM_UL), wartością jej pełna nazwa.
df_ulic.dropna(subset=['SYM_UL', 'NAZWA_1'], inplace=True)
street_dictionary = pd.Series(df_ulic.NAZWA_1.values, index=df_ulic.SYM_UL).to_dict()
print(f"Utworzono słownik z {len(street_dictionary)} unikalnymi ulicami.")

# --- Krok 2b: Czyszczenie i inżynieria cech ---
df_processed = df_main.copy()
print(f"\nRozmiar początkowy: {df_processed.shape}")

# Podstawowe czyszczenie
df_processed.dropna(subset=['Area', 'Price', 'Location', 'Description'], inplace=True)
print(f"Rozmiar po usunięciu NaN: {df_processed.shape}")

# --- Krok 2c: Inteligentna ekstrakcja Dzielnicy i Ulicy ---
def process_location(row):
    location_str = row['Location']
    if not isinstance(location_str, str):
        return pd.Series([np.nan, np.nan])

    parts = [p.strip() for p in location_str.split(',')]
    district = np.nan
    street = np.nan

    # Ekstrakcja dzielnicy
    if len(parts) >= 3:
        district = parts[2]
    
    # Ekstrakcja ulicy (priorytet dla słownika TERYT)
    if pd.notna(row['StreetNumber']):
        try:
            street_sym = int(row['StreetNumber'])
            if street_sym in street_dictionary:
                street = street_dictionary[street_sym]
        except (ValueError, TypeError):
            pass
    
    # Jeśli słownik zawiódł, próbuj z tekstu
    if pd.isna(street) and len(parts) >= 4:
        street = parts[3]
    
    # Czyszczenie nazwy ulicy
    if isinstance(street, str):
        street = re.sub(r'^(ul\.|al\.|al\.|pl\.)\s*', '', street, flags=re.IGNORECASE).lower()
        if len(street) < 3: street = np.nan # Odrzuć bardzo krótkie nazwy
            
    return pd.Series([district, street])

# Zastosuj funkcję i stwórz nowe kolumny
df_processed[['District', 'Ulica_clean']] = df_processed.apply(process_location, axis=1)

# Usuń wiersze, gdzie nie udało się zidentyfikować ulicy
df_processed.dropna(subset=['Ulica_clean'], inplace=True)
print(f"Po ekstrakcji i czyszczeniu ulic -> Pozostało wierszy: {len(df_processed)}, Unikalnych ulic: {df_processed['Ulica_clean'].nunique()}")

# --- Krok 2d: Ostatnie przygotowania ---
# Konwersja daty
df_processed['BuiltYear'] = pd.to_datetime(df_processed['BuiltYear'], format='%Y', errors='coerce')

# Usunięcie outlierów (teraz, na czystym zbiorze)
q_low = df_processed["Price"].quantile(0.01)
q_hi  = df_processed["Price"].quantile(0.99)
df_processed = df_processed[(df_processed["Price"] < q_hi) & (df_processed["Price"] > q_low)]
print(f"Po usunięciu outlierów cenowych -> Pozostało wierszy: {len(df_processed)}")

df_final = df_processed.copy()

KeyError: ['SYM_UL', 'NAZWA_1']

In [ ]:
# === SEKCJA 3: OSTATECZNE PRZYGOTOWANIE I SETUP ===

# --- Krok 3a: Filtrowanie rzadkich ULIC ---
MIN_SAMPLES_PER_STREET = 15 # Zaczynamy z solidnym progiem
street_counts = df_final['Ulica_clean'].value_counts()
streets_to_remove = street_counts[street_counts < MIN_SAMPLES_PER_STREET].index
df_model_ready = df_final[~df_final['Ulica_clean'].isin(streets_to_remove)].copy()
print(f"Po odfiltrowaniu rzadkich ulic -> Pozostało wierszy: {len(df_model_ready)}, Unikalnych ulic: {df_model_ready['Ulica_clean'].nunique()}")

# --- Krok 3b: Podział na zbiory i przygotowanie grup ---
data_train = df_model_ready.sample(frac=0.9, random_state=1122)
data_test = df_model_ready.drop(data_train.index)
fold_groups = data_train['Ulica_clean'] # Do stabilnej walidacji
print(f"Podział danych -> Treningowe: {data_train.shape}, Testowe: {data_test.shape}")

# --- Krok 3c: Setup PyCaret ---
# Definicja cech. Teraz dołączamy dzielnicę jako predyktor!
categorical_features = ['District', 'BuildingType', 'TypeOfMarket']
# PyCaret sam sobie poradzi z resztą (numeryczne, daty)

s = setup(
    data=data_train,
    test_data=data_test,
    target='Ulica_clean',
    session_id=1122,
    n_jobs=4,
    
    # Używamy stabilizacji przez grupowanie
    fold_strategy='groupkfold',
    fold_groups=fold_groups,
    
    # Jawnie definiujemy tylko najważniejsze cechy kategoryczne
    categorical_features=categorical_features,
    
    # Ignorujemy wszystko, co jest ID lub surowym tekstem
    ignore_features=['SaleId', 'OriginalId', 'Location', 'Description', 'Title', 'Link'],
    
    # Zabezpieczenia
    log_experiment=False,
    html=False,
    verbose=True
)

In [ ]:
# === SEKCJA 4: PRZYGOTOWANIE DO MODELOWANIA (ULEPSZONE) ===

# --- Krok 4a: Bardziej Agresywne Filtrowanie ---
MIN_SAMPLES_PER_STREET = 15 # Zwiększamy próg, aby zapewnić stabilność
street_counts = df_final['Ulica_clean'].value_counts()
streets_to_remove = street_counts[street_counts < MIN_SAMPLES_PER_STREET].index
df_model_ready = df_final[~df_final['Ulica_clean'].isin(streets_to_remove)].copy()

print(f"Użyto progu MIN_SAMPLES_PER_STREET = {MIN_SAMPLES_PER_STREET}")
print(f"Usunięto {len(streets_to_remove)} rzadkich ulic. Pozostało {len(df_model_ready)} wierszy.")
print(f"Liczba klas (ulic) do predykcji: {df_model_ready['Ulica_clean'].nunique()}")

# --- Krok 4b: Podział na zbiór treningowy i testowy ---
data_train = df_model_ready.sample(frac=0.9, random_state=1122)
data_test = df_model_ready.drop(data_train.index)
print(f"\nPodział danych -> Treningowe: {data_train.shape}, Testowe: {data_test.shape}")

# --- Krok 4c (NOWOŚĆ): Przygotowanie grup do walidacji krzyżowej ---
# Tworzymy kolumnę, która będzie używana do grupowania foldów.
# Dzięki temu wszystkie wiersze z tą samą ulicą trafią zawsze do tego samego foldu.
fold_groups = data_train['Ulica_clean']
print("\nPrzygotowano grupy do stabilnej walidacji krzyżowej.")

In [ ]:
# === SEKCJA 5: SETUP I TRENING (ULEPSZONE) ===

# Definicje list cech (bez zmian)
# ... (kopiuj kod z definicjami numeric_features, categorical_features, etc. z poprzedniej wersji)
numeric_features = [c for c in data_train.columns if c.startswith('des_tfidf_')] + ['Area', 'Price', 'NumberOfRooms', 'Floor', 'Floors']
categorical_features = ['BuildingType', 'TypeOfMarket', 'District'] 
date_features = ['BuiltYear']
numeric_features = [f for f in numeric_features if f in data_train.columns]
categorical_features = [f for f in categorical_features if f in data_train.columns]
date_features = [f for f in date_features if f in data_train.columns]
if 'BuiltYear' in date_features:
    data_train['BuiltYear'] = pd.to_datetime(data_train['BuiltYear'], errors='coerce')
    data_test['BuiltYear'] = pd.to_datetime(data_test['BuiltYear'], errors='coerce')
for col in categorical_features:
    data_train[col] = data_train[col].astype(str).fillna('missing')
    data_test[col] = data_test[col].astype(str).fillna('missing')
ignore_features = [col for col in ['SaleId', 'OriginalId', 'Location', 'Description', 'Title', 'Link'] if col in data_train.columns]


# Setup
s = setup(
    data=data_train,
    test_data=data_test,
    target='Ulica_clean',
    session_id=1122,
    log_experiment=False,
    n_jobs=4,
    
    # === KLUCZOWA ZMIANA: Używamy grupowania, aby ustabilizować walidację ===
    fold_strategy='groupkfold',
    fold_groups=fold_groups,
    
    numeric_features=numeric_features,
    categorical_features=categorical_features,
    date_features=date_features,
    ignore_features=ignore_features,
    
    html=False,
    verbose=True
)

In [ ]:
# === SEKCJA 6: TRENING I LOGOWANIE DO MLFLOW (ULEPSZONE) ===

if mlflow.active_run():
    mlflow.end_run()

with mlflow.start_run(run_name="Warsaw_Street_Prediction_Run_v2") as run:
    print(f"Rozpoczęto run w MLflow: {run.info.run_id}")
    print("Rozpoczynam porównywanie modeli (ze stabilizacją i modelem dummy)...")
    
    # Dodajemy 'dummy' do listy, aby mieć punkt odniesienia
    models_to_try = ['lightgbm', 'xgboost', 'rf', 'dummy']
    
    best_model = compare_models(include=models_to_try, sort='F1')
    
    print("\nPorównanie modeli zakończone.")
    
    all_models_metrics = pull()
    display(all_models_metrics)
    
    if not all_models_metrics.empty:
        all_models_metrics.to_csv("compare_models_results_v2.csv")
        mlflow.log_artifact("compare_models_results_v2.csv")
        print(f"\nNajlepszy zidentyfikowany model: {best_model}")
    else:
        print("\nUWAGA: Żaden z testowanych modeli nie zakończył treningu pomyślnie.")